## Neo4j Import of GraphRAG Result Parquet files
This notebook imports the results of the GraphRAG indexing process into the Neo4j Graph database for further processing, analysis or visualization.

### How does it work?
The notebook loads the parquet files from the output folder of your indexing process and loads them into Pandas dataframes. It then uses a batching approach to send a slice of the data into Neo4j to create nodes and relationships and add relevant properties. The id-arrays on most entities are turned into relationships.

All operations use `MERGE`, so they are idempotent, and you can run the script multiple times.

If you need to clean out the database, you can run the following statement
```
MATCH (n)
CALL { WITH n DETACH DELETE n } IN TRANSACTIONS OF 25000 ROWS;
```

In [4]:
GRAPHRAG_FOLDER="beautyrag_new_prompts/output"

In [5]:
import pandas as pd
from neo4j import GraphDatabase
import time
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)

True

In [7]:
# NEO4J_URI="bolt://localhost"
# NEO4J_USERNAME="neo4j"
# NEO4J_PASSWORD="password"
# NEO4J_DATABASE="neo4j"

NEO4J_URI=os.getenv("NEO4J_CONNECTION_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE="neo4j"

# print(NEO4J_URI)
# print(NEO4J_PASSWORD)

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [9]:
def batched_import(statement, df, batch_size=1000):
    """
    Import a dataframe into Neo4j using a batched approach.
    Parameters: statement is the Cypher query to execute, df is the dataframe to import, and batch_size is the number of rows to import in each batch.
    """
    total = len(df)
    start_s = time.time()
    for start in range(0,total, batch_size):
        batch = df.iloc[start: min(start+batch_size,total)]
        result = driver.execute_query("UNWIND $rows AS value " + statement, 
                                      rows=batch.to_dict('records'),
                                      database_=NEO4J_DATABASE)
        print(result.summary.counters)
    print(f'{total} rows in { time.time() - start_s} s.')    
    return total

### Indexes and Constraints
Indexes in Neo4j are only used to find the starting points for graph queries, e.g. quickly finding two nodes to connect. Constraints exist to avoid duplicates, we create them mostly on id's of Entity types.

We use some Types as markers with two underscores before and after to distinguish them from the actual entity types.

The default relationship type here is `RELATED` but we could also infer a real relationship-type from the description or the types of the start and end-nodes.

* `__Entity__`
* `__Document__`
* `__Chunk__`
* `__Community__`
* `__Covariate__`

In [10]:
# create constraints, idempotent operation

statements = """
create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique;
create constraint document_id if not exists for (d:__Document__) require d.id is unique;
create constraint entity_id if not exists for (c:__Community__) require c.community is unique;
create constraint entity_id if not exists for (e:__Entity__) require e.id is unique;
create constraint entity_title if not exists for (e:__Entity__) require e.name is unique;
create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique;
create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique;
""".split(";")

for statement in statements:
    if len((statement or "").strip()) > 0:
        print(statement)
        driver.execute_query(statement)


create constraint chunk_id if not exists for (c:__Chunk__) require c.id is unique

create constraint document_id if not exists for (d:__Document__) require d.id is unique

create constraint entity_id if not exists for (c:__Community__) require c.community is unique

create constraint entity_id if not exists for (e:__Entity__) require e.id is unique

create constraint entity_title if not exists for (e:__Entity__) require e.name is unique

create constraint entity_title if not exists for (e:__Covariate__) require e.title is unique

create constraint related_id if not exists for ()-[rel:RELATED]->() require rel.id is unique


## Import Process
### Importing the Documents
We're loading the parquet file for the documents and create nodes with their ids and add the title property. We don't need to store text_unit_ids as we can create the relationships and the text content is also contained in the chunks.

In [11]:
# doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/documents.parquet', columns=["id", "title"])
doc_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/documents.parquet', columns=["id","human_readable_id","title","text","text_unit_ids","creation_date","metadata"])
print(doc_df.head(2))
print(doc_df.shape)

                                                  id  human_readable_id  \
0  f3c61ded8ac66a3a94b8213793b17ff4b7f29077d1fab8...                  1   

              title                                               text  \
0  unstructured.txt  Product Type: Moisturizer\nBrand: SK-II\nProdu...   

                                       text_unit_ids  \
0  [be848e919a7d5dc0fed178abbb82e3031fb31a56bd3db...   

               creation_date metadata  
0  2025-04-21 15:11:09 -0400     None  
(1, 7)


In [12]:
# import documents
statement = """
MERGE (d:__Document__ {id:value.id})
SET d += value {.title}
"""

batched_import(statement, doc_df)

{'_contains_updates': True, 'labels_added': 1, 'nodes_created': 1, 'properties_set': 2}
1 rows in 0.3164370059967041 s.


1

### Loading Text Units
We load the text units, create a node per id and set the text and number of tokens. Then we connect them to the documents that we created before.

In [13]:
text_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/text_units.parquet',
                        columns=["id","text","n_tokens","document_ids"])
text_df.head(2)
# ,id,human_readable_id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids

,id,text,n_tokens,document_ids
0,be848e919a7d5dc0fed178abbb82e3031fb31a56bd3dbd...,Product Type: Moisturizer\nBrand: SK-II\nProdu...,1200,[f3c61ded8ac66a3a94b8213793b17ff4b7f29077d1fab...
1,3f98d135f480411fb44a62c59676accf0ff8fb74d03350...,"aniol, Ci 77891 (Titanium Dioxide).\nFor Skin ...",1200,[f3c61ded8ac66a3a94b8213793b17ff4b7f29077d1fab...


In [14]:
statement = """
MERGE (c:__Chunk__ {id:value.id})
SET c += value {.text, .n_tokens}
WITH c, value
UNWIND value.document_ids AS document
MATCH (d:__Document__ {id:document})
MERGE (c)-[:PART_OF]->(d)
"""

batched_import(statement, text_df)

{'_contains_updates': True, 'labels_added': 8, 'relationships_created': 8, 'nodes_created': 8, 'properties_set': 24}
8 rows in 0.3524789810180664 s.


8

### Loading Nodes
For the nodes we store id, name, description, embedding (if available), human readable id.

In [15]:
entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/entities.parquet',
                            columns=["title","type","description","human_readable_id","id","text_unit_ids"])
#                             columns=["name","type","description","human_readable_id","id","description_embedding","text_unit_ids"])
embeddings_entity_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/embeddings.entity.description.parquet',
                            columns=["embedding"])
embeddings_entity_df.rename(columns={"embedding": "description_embedding"}, inplace=True)
entity_df = pd.concat([entity_df, embeddings_entity_df], axis=1)
entity_df.head(2)
# ,id,human_readable_id,title,type,description,text_unit_ids,
# ,id,embedding

,title,type,description,human_readable_id,id,text_unit_ids,description_embedding
0,MOISTURIZER,PRODUCT TYPE,MOISTURIZER refers to a category of skincare p...,0,8824eb45-1eeb-4e73-81c1-62b3b19d5543,[be848e919a7d5dc0fed178abbb82e3031fb31a56bd3db...,"[0.013597052544355392, 0.0408736877143383, -0...."
1,SK-II,BRAND,SK-II is a luxury skincare brand renowned for ...,1,cfaa854b-aa57-413e-83de-7b8dcba0c9b9,[be848e919a7d5dc0fed178abbb82e3031fb31a56bd3db...,"[0.012267841026186943, -0.0031117114704102278,..."


In [16]:
entity_statement = """
MERGE (e:__Entity__ {id:value.id})
SET e += value {.human_readable_id, .description, name:replace(value.name,'"','')}
WITH e, value
CALL db.create.setNodeVectorProperty(e, "description_embedding", value.description_embedding)
CALL apoc.create.addLabels(e, case when coalesce(value.type,"") = "" then [] else [apoc.text.upperCamelCase(replace(value.type,'"',''))] end) yield node
UNWIND value.text_unit_ids AS text_unit
MATCH (c:__Chunk__ {id:text_unit})
MERGE (c)-[:HAS_ENTITY]->(e)
"""

batched_import(entity_statement, entity_df)

{'_contains_updates': True, 'labels_added': 389, 'relationships_created': 601, 'nodes_created': 389, 'properties_set': 1556}
389 rows in 4.710721015930176 s.


389

### Import Relationships
For the relationships we find the source and target node by name, using the base `__Entity__` type. After creating the RELATED relationships, we set the description as attribute.

In [17]:
# rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/relationships.parquet',
#                         columns=["source","target","id","rank","weight","human_readable_id","description","text_unit_ids"])
rel_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/relationships.parquet',
                        columns=["source","target","id","weight","human_readable_id","description","text_unit_ids"])

rel_df.head(2)
# ,id,human_readable_id,source,target,description,weight,combined_degree,text_unit_ids
# RANK OMMITTED NOW

,source,target,id,weight,human_readable_id,description,text_unit_ids
0,SK-II,FACIAL TREATMENT ESSENCE,0cca2682-de40-465c-a040-d771f7437091,9.0,0,SK-II is the brand that produces the Facial Tr...,[be848e919a7d5dc0fed178abbb82e3031fb31a56bd3db...
1,SK-II,GENOPTICS AURA ESSENCE SERUM,5149b6a3-13dc-4b4e-bf49-3bc782554601,9.0,1,SK-II is the brand that produces the GenOptics...,[be848e919a7d5dc0fed178abbb82e3031fb31a56bd3db...


In [18]:
# ORIGINAL: 
# rel_statement = """
#     MATCH (source:__Entity__ {name:replace(value.source,'"','')})
#     MATCH (target:__Entity__ {name:replace(value.target,'"','')})
#     // not necessary to merge on id as there is only one relationship per pair
#     MERGE (source)-[rel:RELATED {id: value.id}]->(target)
#     SET rel += value {.rank, .weight, .human_readable_id, .description, .text_unit_ids}
#     RETURN count(*) as createdRels
# """
rel_statement = """
    MATCH (source:__Entity__ {name:replace(value.source,'"','')})
    MATCH (target:__Entity__ {name:replace(value.target,'"','')})
    // not necessary to merge on id as there is only one relationship per pair
    MERGE (source)-[rel:RELATED {id: value.id}]->(target)
    SET rel += value {.weight, .human_readable_id, .description, .text_unit_ids}
    RETURN count(*) as createdRels
"""

batched_import(rel_statement, rel_df)

{}
202 rows in 0.43020129203796387 s.


202

### Importing Communities
For communities we import their id, title, level. We connect the `__Community__` nodes to the start and end nodes of the relationships they refer to.

Connecting them to the chunks they orignate from is optional, as the entites are already connected to the chunks.

In [19]:
community_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/communities.parquet', 
                    columns=["id","level","title","text_unit_ids","relationship_ids"])

community_df.head(2)
# ,id,human_readable_id,community,level,parent,children,title,entity_ids,relationship_ids,text_unit_ids,period,size

,id,level,title,text_unit_ids,relationship_ids
0,2ae0808b-bfc6-4a18-8ba8-2777768e003c,0,Community 0,[be848e919a7d5dc0fed178abbb82e3031fb31a56bd3db...,"[1da77834-d501-4e45-8cf9-eacf48d99fb8, 2232a5e..."
1,d9bb24ce-6cf0-4b3f-a30c-1a2b13c5a3c9,0,Community 1,[2a76834194f157ccb568740fa2a34f4641397ee4361da...,"[07d67319-8a76-4fd5-980f-083c2b8c10b9, 08babd0..."


In [20]:
statement = """
MERGE (c:__Community__ {community:value.id})
SET c += value {.level, .title}
/*
UNWIND value.text_unit_ids as text_unit_id
MATCH (t:__Chunk__ {id:text_unit_id})
MERGE (c)-[:HAS_CHUNK]->(t)
WITH distinct c, value
*/
WITH *
UNWIND value.relationship_ids as rel_id
MATCH (start:__Entity__)-[:RELATED {id:rel_id}]->(end:__Entity__)
MERGE (start)-[:IN_COMMUNITY]->(c)
MERGE (end)-[:IN_COMMUNITY]->(c)
RETURN count(distinct c) as createdCommunities
"""

batched_import(statement, community_df)

{'_contains_updates': True, 'labels_added': 20, 'nodes_created': 20, 'properties_set': 60}
20 rows in 0.4274129867553711 s.


20

### Importing Community Reports
Fo the community reports we create nodes for each communitiy set the id, community, level, title, summary, rank, and rank_explanation and connect them to the entities they are about. For the findings we create the findings in context of the communities.

In [21]:
# community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/community_reports.parquet',
#                                 columns=["id","community","level","title","summary", "findings","rank","rank_explanation","full_content"])
community_report_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/community_reports.parquet',
                                columns=["id","community","level","title","summary", "findings","rank","full_content"])
community_report_df.head(2)
# ,id,human_readable_id,community,level,parent,children,title,summary,full_content,rank,rating_explanation,findings,full_content_json,period,size
# RANK_EXPLANATION OMMITTED

,id,community,level,title,summary,findings,rank,full_content
0,0815256a3e6849eb97643b167b59313d,6,1,1.0 OZ Skincare Community,The community centers around high-end skincare...,[{'explanation': 'The entity '1.0 OZ (30ML)' r...,4.0,# 1.0 OZ Skincare Community\n\nThe community c...
1,80c7a5fb05944b53bb0fc91c7d6ccba3,7,1,The Moisturizing Soft Cream and Its Ingredients,The community centers around The Moisturizing ...,[{'explanation': 'The Moisturizing Soft Cream ...,7.5,# The Moisturizing Soft Cream and Its Ingredie...


In [22]:
# import communities
community_statement = """
MERGE (c:__Community__ {community:value.community})
SET c += value {.level, .title, .rank, .full_content, .summary}
WITH c, value
UNWIND range(0, size(value.findings)-1) AS finding_idx
WITH c, value, finding_idx, value.findings[finding_idx] as finding
MERGE (c)-[:HAS_FINDING]->(f:Finding {id:finding_idx})
SET f += finding
"""
batched_import(community_statement, community_report_df)

{'_contains_updates': True, 'labels_added': 119, 'relationships_created': 99, 'nodes_created': 119, 'properties_set': 417}
20 rows in 0.36308789253234863 s.


20

### Importing Covariates
Covariates are for instance claims on entities, we connect them to the chunks where they originate from.

**By default, covariates are not included in the output, so the file might not exists in your output if you didn't set the configuration to extract claims**

In [23]:
"""
# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')
# cov_df.head(2)
"""

"\n# cov_df = pd.read_parquet(f'{GRAPHRAG_FOLDER}/create_final_covariates.parquet')\n# cov_df.head(2)\n"

In [24]:
cov_statement = """
MERGE (c:__Covariate__ {id:value.id})
SET c += apoc.map.clean(value, ["text_unit_id", "document_ids", "n_tokens"], [NULL, ""])
WITH c, value
MATCH (ch:__Chunk__ {id: value.text_unit_id})
MERGE (ch)-[:HAS_COVARIATE]->(c)
"""
# batched_import(cov_statement, cov_df)